In [2]:
import spotipy
import spotipy.util as util
import pandas as pd
import os
import time
import requests
from bs4 import BeautifulSoup as bs
import datetime
from datetime import timedelta, date
import glob
from os import listdir
import numpy as np
import pymongo
import csv
import json
import sys, getopt, pprint
from pymongo import MongoClient
import types
import itertools
from itertools import zip_longest
import scipy.stats
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

In [3]:
DaveDF = pd.read_csv('dfs/DaveDF.csv')
VladDF = pd.read_csv('dfs/VLAD_DF.csv')
KevDF = pd.read_csv('dfs/KevDF.csv')
StephenDF = pd.read_csv('dfs/StephenDF.csv')
WhitePandas = pd.read_csv('dfs/WhitePandas.csv')
DaveDF_means = pd.read_csv('dfs/Dave_means.csv')
VladDF_means = pd.read_csv('dfs/Vlad_means.csv')
KevDF_means = pd.read_csv('dfs/Kev_means.csv')
StephenDF_means = pd.read_csv('dfs/Stephen_means.csv')
WhitePandas_mean = pd.read_csv('dfs/WhitePandas_means.csv')

In [5]:
#bright=["#F8766D","#00BFC4","#FFC400","#03ED3A","#003FFF","#8A2BE2"]
#sns.set_palette(bright)
#
##plot mean audio features by owner
#new = WhitePandas_mean.melt('owner', var_name='cols',  value_name='vals')
#sns.factorplot(x="cols", y="vals", hue='owner', data=new, kind='bar', size=10, legend_out=False)
#plt.xticks(rotation = 90, fontsize = 12) #set audio feature labels
#plt.ylabel("Value", fontsize = 15) #set y axis label
#plt.xlabel("Audio Features", fontsize = 15) #set x axis label
#plt.title("Mean Audio Features by Owner", fontsize = 17) #set chart title
#plt.legend(fontsize=12) #increase legend fontsize
#plt.show() #remove text output

In [19]:
#####Separate Training Set into Train and Test Sets
#separate the dataset into two separate sets to train and test the model
train = pd.read_csv("dfs/WhitePandas.csv")
features = train.drop(['Owner','Track ID','Tracks','Artists', 'Albums'], axis=1)
target = train['Owner']

x_train, x_test, y_train, y_test = train_test_split(features, target, test_size= 0.15)
print('Features Training Set:', x_train.shape, 'Features Testing Set:', x_test.shape)
print('Target Training Set:', y_train.shape, 'Target Testing Set:', y_test.shape)

Features Training Set: (170, 11) Features Testing Set: (30, 11)
Target Training Set: (170,) Target Testing Set: (30,)


In [24]:
#Random Forest
randomforest = RandomForestClassifier()
randomforest.fit(x_train, y_train)
y_pred = randomforest.predict(x_test)
acc_randomforest = accuracy_score(y_pred, y_test) * 100
print("Random Forest:", acc_randomforest)

Random Forest: 76.66666666666667


In [29]:
param_grid = { 
    'n_estimators': [800, 2800],
    'max_features': ['auto', 'sqrt', 'log2']
}

rf_cv = GridSearchCV(estimator=randomforest, param_grid=param_grid)
rf_cv.fit(x_train, y_train)

print(rf_cv.best_params_)

y_pred = rf_cv.predict(x_test)
acc_rfcv = accuracy_score(y_pred, y_test)*100
print("Random Forest GridSearchCV:", acc_rfcv)

{'max_features': 'auto', 'n_estimators': 2800}
Random Forest GridSearchCV: 73.33333333333333


In [30]:
testdf = pd.read_csv('dfs/features.csv')
owners = ['Dave','Kev','Stephen','Vlad','Dave','Kev','Stephen','Vlad',
          'Dave','Kev','Stephen','Vlad','Dave','Kev','Stephen','Vlad',
          'Dave','Kev','Stephen','Vlad','Dave','Kev','Stephen','Vlad',
          'Dave','Kev','Stephen','Vlad','Dave','Kev','Stephen','Vlad',
          'Dave','Kev','Stephen','Vlad','Dave','Kev','Stephen','Vlad',
          'Dave','Kev','Stephen','Vlad','Dave','Kev','Stephen','Vlad',
          'Dave','Kev','Stephen','Vlad','Dave','Kev','Stephen','Vlad',
          'Dave','Kev','Stephen','Vlad','Dave','Kev','Stephen','Vlad',
          'Dave','Kev','Stephen','Vlad','Dave','Kev','Stephen','Vlad',
          'Dave','Kev','Stephen','Vlad','Dave','Kev','Stephen','Vlad',
          'Dave','Kev','Stephen','Vlad','Dave','Kev','Stephen','Vlad',
          'Dave','Kev','Stephen','Vlad','Dave','Kev','Stephen','Vlad',          
          'Dave','Kev','Stephen','Vlad','Dave','Kev','Stephen','Vlad',
          'Dave','Kev','Stephen','Vlad','Dave','Kev','Stephen','Vlad',
          'Dave','Kev','Stephen','Vlad','Dave','Kev','Stephen','Vlad',
          'Dave','Kev','Stephen','Vlad','Dave','Kev','Stephen','Vlad',
          'Dave','Kev','Stephen','Vlad','Dave','Kev','Stephen','Vlad',
          'Dave','Kev','Stephen','Vlad','Dave','Kev','Stephen','Vlad',
          'Dave','Kev','Stephen','Vlad','Dave','Kev','Stephen','Vlad',
          'Dave','Kev','Stephen','Vlad','Dave','Kev','Stephen','Vlad',
          'Dave','Kev','Stephen','Vlad','Dave','Kev','Stephen','Vlad',
          'Dave','Kev','Stephen','Vlad','Dave','Kev','Stephen','Vlad',
          'Dave','Kev','Stephen','Vlad','Dave','Kev','Stephen','Vlad',
          'Dave','Kev','Stephen','Vlad','Dave','Kev','Stephen','Vlad',
          'Dave','Kev','Stephen','Vlad','Dave','Kev','Stephen','Vlad']
prediction = rf_cv.predict(testdf)
accuracy = accuracy_score(prediction, owners)
print("Correct Predictions: %d/%d"%(accuracy*len(owners),len(owners)))
print("Accuracy: %.2f%%"%(accuracy*100))

Correct Predictions: 49/200
Accuracy: 24.50%


In [8]:
###seeing if code was breaking because of too many owners
testdf = pd.DataFrame(columns=['Owner','Track ID', 'Tracks', 
                                    'Artists', 'Albums', 'danceability', 
                                    'energy','key', 'loudness', 'mode', 
                                    'speechiness', 'acousticness',
                                    'instrumentalness', 'liveness', 
                                    'valence', 'tempo'])

for index, row in DaveDF.iterrows():
    testdf.loc[testdf.shape[0]] =  ['Dave', row['Track ID'], 
                                         row['Tracks'], row['Artists'],
                                         row['Albums'], row['danceability'], 
                                         row ['energy'],row['key'], 
                                         row['loudness'], row['mode'], 
                                         row['speechiness'],row['acousticness'], 
                                         row['instrumentalness'], row['liveness'],
                                         row['valence'], row['tempo']]

for index, row in KevDF.iterrows():
    testdf.loc[testdf.shape[0]] =  ['Kev', row['Track ID'], 
                                         row['Tracks'], row['Artists'],
                                         row['Albums'], row['danceability'], 
                                         row ['energy'],row['key'], 
                                         row['loudness'], row['mode'], 
                                         row['speechiness'],row['acousticness'], 
                                         row['instrumentalness'], row['liveness'],
                                         row['valence'], row['tempo']]

testdf = testdf[['danceability','energy', 'key', 'loudness', 'mode', 'speechiness', 
                 'acousticness','instrumentalness', 'liveness', 'valence', 'tempo']]

testdf.to_csv('dfs/testdf.csv',index=False)